# 1. Imports

## 1.1 Libraries 

In [174]:
import re
import sqlite3
import inflection


import pandas   as pd
import numpy    as np

from sqlalchemy import create_engine

## 1.2 Loading Data

In [155]:
hm_data_raw = pd.read_csv('../data/hm_data_raw.csv', dtype={'product_id': str, 'style_id': str, 'color_id': str})

In [156]:
df1 = hm_data_raw.copy()

# 2. Data Understanding

## 2.1 Overview

In [157]:
df2 = df1.copy()

In [158]:
df2.head()

,product_id,product_name,product_category,product_price,style_id,color_id,scrapy_datetime,color_name,Fit,Composition
0,0985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,0985159,001,2022-09-23 09:43:57,Black,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,0985159,001,2022-09-23 09:43:57,Black,Skinny fit,Pocket lining: Cotton 100%
2,0985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,0985159,001,2022-09-23 09:43:57,Black,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,1024256,001,2022-09-23 09:43:57,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,1024256,001,2022-09-23 09:43:57,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


In [159]:
# Rename Columns
cols_old = ['product_id', 'product_name', 'product_category', 'product_price', 'style_id', 'color_id', 'scrapy_datetime', 'color_name', 'Fit', 'Composition']
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df2.columns = cols_new

In [160]:
# H&M Data Dimentions
print(f'Number of Rows:{df2.shape[0]}')
print(f'Number of Columns:{df2.shape[1]}')

Number of Rows:226
Number of Columns:10


In [185]:
df2.apply(lambda x: len(x.unique()))

product_id          35
product_name         8
product_category     6
product_price        2
style_id             8
color_id            24
scrapy_datetime      1
color_name          14
fit                  4
cotton               7
polyester            4
spandex              3
dtype: int64

In [161]:
# Data Types
df2.dtypes

product_id          object
product_name        object
product_category    object
product_price       object
style_id            object
color_id            object
scrapy_datetime     object
color_name          object
fit                 object
composition         object
dtype: object

## 2.2 Data Cleaning

### 2.2.1 Check & Fillout NA 

In [162]:
df2.isna().sum()

product_id          0
product_name        0
product_category    0
product_price       0
style_id            0
color_id            0
scrapy_datetime     0
color_name          0
fit                 0
composition         0
dtype: int64

### 2.2.2 Change Types and Format

In [163]:
# Product ID
# Didn't need to be change.

# Product Name
df2['product_name'] = df2['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# Product Price
df2['product_price'] = df2['product_price'].apply(lambda x: x.replace('$', '')).astype(float)

# Style ID
df2['style_id'] = df2['style_id'].astype(int)

# Color ID
df2['color_id'] = df2['color_id'].astype(int)

# Scrapy Datetime
df2['scrapy_datetime'] = pd.to_datetime(df2['scrapy_datetime'], format = '%Y-%m-%d %H:%M:%S')

# Color Name
df2['color_name'] = df2['color_name'].apply(lambda x: x.replace(' ', '_').lower())

# Fit
df2['fit'] = df2['fit'].apply(lambda x: x.replace(' ', '_').lower())

### 2.2.3 Cleaning Composition Column

In [164]:
# Drop Composition "Pants Parts"
df2 = df2[~df2['composition'].str.contains('Pocket', na=False)]
df2 = df2[~df2['composition'].str.contains('Shell', na=False)]
df2 = df2[~df2['composition'].str.contains('Lining', na=False)]
df2 = df2.reset_index(drop=True)

# Break composition by comma
aux = df2['composition'].str.split(',', expand=True)

# Sequence composition (cotton | polyester | Spandex):
df_ref = pd.DataFrame(index = np.arange(len(df2)), columns = ['cotton', 'polyester', 'spandex'])

# Cotton 
df_cotton = aux[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# Polyester
df_polyester = aux.loc[aux[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

# Spandex
df_spandex = aux.loc[aux[1].str.contains('Spandex', na=True), 1]
df_spandex.name = 'spandex'
df_spandex = df_spandex.combine_first(aux[2])       # combine_first function merge the 2 columns that contains spandex information

#Other way to merge:
# df_spandex = pd.concat([df_spandex, aux[2]], axis = 1)
# df_spandex = df_spandex.sort_index()
# df_spandex['spandex'] = df_spandex.apply(lambda x: x[1] if (pd.notnull(x[1])) else x[2], axis = 1)
# df_spandex = df_spandex['spandex']

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')

# Final Join
df2 = pd.concat ([df2, df_ref], axis = 1)

# Format Composition Data
df2['cotton'] = df2['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df2['polyester'] = df2['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df2['spandex'] = df2['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

# Drop Columns
df2 = df2.drop(columns = ['composition'], axis=1)


In [165]:
df2.shape

(54, 12)

In [166]:
df2.head()

,product_id,product_name,product_category,product_price,style_id,color_id,scrapy_datetime,color_name,fit,cotton,polyester,spandex
0,0938875007,slim_tapered_jeans,men_jeans_slim,39.99,938875,7,2022-09-23 09:43:57,black,slim_fit,0.99,0.00,0.01
1,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.80,0.19,0.01
2,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.77,0.21,0.02
3,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.78,0.21,0.01
4,1004199007,skinny_cropped_jeans,men_jeans_skinny,29.99,1004199,7,2022-09-23 09:43:57,denim_blue,skinny_fit,0.99,0.00,0.01


In [167]:
df2.apply(lambda x: len(x.unique()))

product_id          35
product_name         8
product_category     6
product_price        2
style_id             8
color_id            24
scrapy_datetime      1
color_name          14
fit                  4
cotton               7
polyester            4
spandex              3
dtype: int64

# 3. Save Changes

## 3.1 Save .csv File

In [168]:
# Save as .csv
df2.to_csv('../data/hm_data_cleaned.csv', index=False)

## 3.2 Create SQL Database 

In [171]:
df2.dtypes

product_id                  object
product_name                object
product_category            object
product_price              float64
style_id                     int64
color_id                     int64
scrapy_datetime     datetime64[ns]
color_name                  object
fit                         object
cotton                     float64
polyester                  float64
spandex                    float64
dtype: object

In [182]:
# Database Conection

endpoint_local = 'sqlite:////home/vitor/Repos/market-research/data/hm_db.sqlite'

engine = create_engine(endpoint_local)
connection = engine.connect()

# Create Table
# query_showroom_schema = """ 
#     CREATE TABLE showroom (
#         product_id                  TEXT,
#         product_name                TEXT,
#         product_category            TEXT,
#         product_price               REAL,
#         style_id                    INTEGER,
#         color_id                    INTEGER,
#         scrapy_datetime             TEXT,
#         color_name                  TEXT,
#         fit                         TEXT,
#         cotton                      REAL,
#         polyester                   REAL,
#         spandex                     REAL
#     )

# """

# cursor = conn.execute(query_showroom_schema)

# conn.commit()
# conn.close()

# Insert Data
df2.to_sql('showroom', con = engine, if_exists='append', index=False)

54

In [183]:
# Consulting Database
query = """ 
    SELECT * FROM showroom

"""

df3 = pd.read_sql_query(query, engine)
df3.head()


,product_id,product_name,product_category,product_price,style_id,color_id,scrapy_datetime,color_name,fit,cotton,polyester,spandex
0,0938875007,slim_tapered_jeans,men_jeans_slim,39.99,938875,7,2022-09-23 09:43:57,black,slim_fit,0.99,0.00,0.01
1,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.80,0.19,0.01
2,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.77,0.21,0.02
3,0993887007,hybrid_regular_denim_joggers,men_jeans_joggers,39.99,993887,7,2022-09-23 09:43:57,black/no_fade_black,regular_fit,0.78,0.21,0.01
4,1004199007,skinny_cropped_jeans,men_jeans_skinny,29.99,1004199,7,2022-09-23 09:43:57,denim_blue,skinny_fit,0.99,0.00,0.01


In [184]:
connection.close()